<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice SQL with `pandas`, Pt. 1

_Authors: Sam Stack (DC)_

---

In [102]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Review: `pandas` and SQL


### The `pandas` Connector and Functions for SQL

We can leverage SQL through `pandas` using the `pandas.io.sql` module:

```python
import pandas as pd
from pandas.io import sql
```


#### Sql.read_sql_table(table_name, con[, schema, ...])
- Reads a SQL database table into a DataFrame.

#### Sql.read_sql_query(sql, con[, index_col, ...])
- Reads a SQL query into a DataFrame.

#### Sql.read_sql(sql, con[, index_col, ...])
- Reads a SQL query or database table into a DataFrame.
- Adds a convenience wrapper around `read_sql_table()` and `read_sql_query()`.
- Delegates to a specific function, depending on the provided input.

#### DataFrame.to_sql(name, con[, flavor, ...])
- Writes records stored in a DataFrame to a SQL database.

---

### 1.  Create a SQL DB and tables using `pandas` DFs and `.csv`s.

First, we will need to read our`.csv` files into Python before we can use it to convert them to a SQL-style DataFrame.

**Now, let's connect to the SQLite database. If no database exists, our command will create one.**

*Keep in mind that the directory your notebook opens is its base directory for all future SQL actions.*

```python
connection = sqlite3.connect('./datasets/sql/Cars.db.sqlite')
```

In [4]:
import pandas as pd
from pandas.io import sql
import sqlite3

connection = sqlite3.connect('./datasets/sql/Cars.db.sqlite')

In [5]:
car_names = pd.read_csv('./datasets/csv/car-names.csv')

#### Convert the loaded `.csv` to a SQL file.  
Because DataFrames are similar to SQL tables, you can now read and convert a `pandas` DataFrame named `car_names` into a SQL table in the newly created SQLite database above.

```python
car_names.to_sql(name = 'car_names', con = connection, if_exists = 'replace', index = False)
```

Important `.to_sql` arguments include:
- `name`: The name of the table; useful if you have multiple tables in a SQL database.
- `con`: The connection path to where the data should be placed.
- `if_exists`: The condition to pass if the table already exists.

If you check that directory now, you should see a `cars.db` SQL file.

In [6]:
car_names.to_sql(name = 'car_names', con = connection, if_exists = 'replace', index = False)

> **Note:** If you wanted a temporary SQL database, using the command below would allow you to access a database store in memory (RAM) as opposed to in storage.

``` python
conn = sqlite3.connect(':memory:')
```

### 2. Create a table in the `cars` database for car makers.

The table should be called `car_makers`.

In [11]:
car_makers = pd.read_csv('./datasets/csv/car-makers.csv')

In [12]:
car_makers.to_sql(name = 'car_makers', con = connection, if_exists = 'replace', index = False)

### 3. Create a table in the `cars` database for the car data.

The table should be called `car_data`.

In [15]:
cars_data = pd.read_csv('./datasets/csv/cars-data.csv')

In [16]:
cars_data.to_sql(name = 'cars_data', con = connection, if_exists = 'replace', index = False)

### 4. Using a query string, read the entire `car_names` table from your SQL database into a DataFrame.

Reading into a DataFrame with a query string can be accomplished using:
```python
# Use `read_sql` from the `pandas` SQL library and set it equal to a DF object.
cars = sql.read_sql(query_string, con = connection)
```

In [24]:
cars = sql.read_sql('SELECT * FROM car_names', con = connection)
cars.head(3)

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'


> **Tip:** If you type `Shift + Tab` in the function call, you can see that the `read_sql` function takes the arguments 'sql' and 'con.'

## Side Note: Normalized vs. Denormalized Databases

---

There are several ways to organize data in a relational database. Two common definitions for data set ups are **normalized** and **denormalized**.

- __Normalized__ structures include a single table per entity and use many foreign keys or link tables to connect entities together.

- __Denormalized__ tables have fewer tables and may (for example) place all of the tweets and user information in one table.

Each style has its own advantages and disadvantages. Denormalized tables duplicate a lot of information. For example, in a combined tweets/users table, we may store the address of each user. Now, instead of storing this once per user, we are storing it once per tweet!

However, this makes the data easy to access if we ever need to find the tweet _and_ the user's location.

Normalized tables save the storage space by separating the information. However, if we ever need to access two pieces of information at once — like in our example — we would need to join the corresponding tables, which can take more time.


### 5. Write a Python function to query a database using `pandas` and return a DataFrame.

The function should take two arguments:
- The query string
- The database connection object

In [25]:
def sqldf(query, connection):
    return sql.read_sql(query, connection)

### 6. Select the first five rows of the `car_names` table.

> **Hint**: The LIMIT command in SQL can limit the number of rows returned.

In [26]:
sqldf('SELECT * FROM car_names LIMIT 5', connection)

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'
3,4,'amc','amc rebel sst'
4,5,'ford','ford torino'


### 7. Add the cars into the `car_names` table.

The `.execute()` function will come in handy here, executing a SQL command string.
```python
connection.execute()
```

In [39]:
ferrari = (None, 'Ferrari','The Ferrari')
tesla = [None, 'Tesla', None]

In [77]:
connection.execute('INSERT INTO car_names VALUES (?,?,?)', ferrari)

In [66]:
connection.execute('INSERT INTO car_names VALUES (?,?,?)', tesla)

In [76]:
connection.execute('DELETE FROM car_names WHERE Model = "Ferrari"')

In [89]:
sqldf('SELECT * FROM car_names LIMIT 5', connection)

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,2,'buick','buick skylark 320'
2,3,'plymouth','plymouth satellite'
3,4,'amc','amc rebel sst'
4,5,'ford','ford torino'


### 8. Query the `car_names` table for all columns where `'Model' = 'Tesla.'`

In [94]:
sqldf('SELECT * FROM car_names WHERE Model = "Tesla"', connection)

,Id,Model,Make
0,None,Tesla,None


In [101]:
sql.read_sql('SELECT * FROM car_names WHERE Model LIKE "%chevrolet%"', connection)

,Id,Model,Make
0,1,'chevrolet','chevrolet chevelle malibu'
1,7,'chevrolet','chevrolet impala'
2,12,'chevrolet','chevrolet chevelle concours (sw)'
3,19,'chevrolet','chevrolet monte carlo'
4,33,'chevrolet','chevy c20'
5,37,'chevrolet','chevrolet vega 2300'
6,43,'chevrolet','chevrolet chevelle malibu'
7,46,'chevrolet','chevrolet impala'
8,54,'chevrolet','chevrolet vega (sw)'
9,68,'chevrolet','chevrolet vega'


### 9. Select the first five rows of the `car_makers` table.

In [98]:
sqldf('SELECT * FROM car_makers LIMIT 5', connection)

,Id,Maker,FullName,Country
0,1,'amc','American Motor Company',1.0
1,2,'volkswagen','Volkswagen',2.0
2,3,'bmw','BMW',2.0
3,4,'gm','General Motors',1.0
4,5,'ford','Ford Motor Company',1.0


### 10. Select the first five rows of the `car_data` table.

In [99]:
sqldf('SELECT * FROM car_data LIMIT 5', connection)

,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year
0,1,18.0,8,307.0,130.0,3504,12.0,1970
1,2,15.0,8,350.0,165.0,3693,11.5,1970
2,3,18.0,8,318.0,150.0,3436,11.0,1970
3,4,16.0,8,304.0,150.0,3433,12.0,1970
4,5,17.0,8,302.0,140.0,3449,10.5,1970


## SQL JOIN Types

---

SQL JOINs are used when data is spread out in different tables. A JOIN operation allows us to combine rows from two or more tables in a single new table. In order for this to be possible, a common field need to exists between the tables.

JOIN operations can be considered operations between two sets in which records with the same key are combined and records missing in one set are either discarded or included as NULL values.

![join types](./assets/images/joins.gif)

JOIN types include:
- **INNER JOIN:** Returns all rows where there is at least one match in BOTH tables.
- **LEFT JOIN:** Returns all rows from the left table and matched rows from the right table.
- **RIGHT JOIN:** Returns all rows from the right table and matched rows from the left table.
- **FULL JOIN:** Returns all rows where there is a match in ONE of the tables.

![sql join types](./assets/images/sql-joins.jpeg)

### In our example, we can use Order ID as the matching feature to perform merges.

Let's check out all of the ways we can merge these tables.

### 11. Practice INNER JOINs

The most common type of JOIN is `SQL INNER JOIN` (SIMPLE JOIN). A `SQL INNER JOIN` returns all rows from multiple tables in which the JOIN condition is met. 

If we `INNER JOIN` on `Id`, it takes the intersection of the two tables, excluding the rows for which `CustomerID` is NULL in EITHER of the two tables.

Essentially, only matching pairs of Order IDs from both data sets will be returned.

**Select `Make`, `MPG`, `Horsepower`, and `Year`**.
- You will need to `INNER JOIN` the `car_names` and `car_data` tables on the `Id` column.


In [113]:
sqldf('SELECT * FROM car_names', connection)

,Id,Model,Make
0,1.0,'chevrolet','chevrolet chevelle malibu'
1,2.0,'buick','buick skylark 320'
2,3.0,'plymouth','plymouth satellite'
3,4.0,'amc','amc rebel sst'
4,5.0,'ford','ford torino'
5,6.0,'ford','ford galaxie 500'
6,7.0,'chevrolet','chevrolet impala'
7,8.0,'plymouth','plymouth fury iii'
8,9.0,'pontiac','pontiac catalina'
9,10.0,'amc','amc ambassador dpl'


In [112]:
sqldf('SELECT * FROM car_data', connection)

,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year
0,1,18.0,8,307.0,130.0,3504,12.0,1970
1,2,15.0,8,350.0,165.0,3693,11.5,1970
2,3,18.0,8,318.0,150.0,3436,11.0,1970
3,4,16.0,8,304.0,150.0,3433,12.0,1970
4,5,17.0,8,302.0,140.0,3449,10.5,1970
5,6,15.0,8,429.0,198.0,4341,10.0,1970
6,7,14.0,8,454.0,220.0,4354,9.0,1970
7,8,14.0,8,440.0,215.0,4312,8.5,1970
8,9,14.0,8,455.0,225.0,4425,10.0,1970
9,10,15.0,8,390.0,190.0,3850,8.5,1970


In [110]:
sqldf('SELECT a.Make, b.MPG, b.Horsepower, b.Year FROM car_names a INNER JOIN car_data b ON a.ID = b.ID LIMIT 5', connection)

,Make,MPG,Horsepower,Year
0,'chevrolet chevelle malibu',18.0,130.0,1970
1,'buick skylark 320',15.0,165.0,1970
2,'plymouth satellite',18.0,150.0,1970
3,'amc rebel sst',16.0,150.0,1970
4,'ford torino',17.0,140.0,1970


### 12. Practice LEFT JOINs

The `LEFT JOIN` keyword returns all rows from the left table (`table1`), along with matching rows in the right table (`table2`). When there is no match, the result is NULL on the right side.

**Select `Make`, `MPG`, `Horsepower`, and `Year`**.
- `SELECT FROM` the `car_names` table.
- `LEFT JOIN` the `car_data` table by `Id`.

In [111]:
sqldf('SELECT a.Make, b.MPG, b.Horsepower, b.Year FROM car_names a LEFT JOIN car_data b ON a.ID = b.ID', connection)

,Make,MPG,Horsepower,Year
0,'chevrolet chevelle malibu',18.0,130.0,1970.0
1,'buick skylark 320',15.0,165.0,1970.0
2,'plymouth satellite',18.0,150.0,1970.0
3,'amc rebel sst',16.0,150.0,1970.0
4,'ford torino',17.0,140.0,1970.0
5,'ford galaxie 500',15.0,198.0,1970.0
6,'chevrolet impala',14.0,220.0,1970.0
7,'plymouth fury iii',14.0,215.0,1970.0
8,'pontiac catalina',14.0,225.0,1970.0
9,'amc ambassador dpl',15.0,190.0,1970.0


###  RIGHT JOINs and FULL OUTER JOINs (Unsupported)

> **Note:** We haven't included exercises for RIGHT and FULL OUTER JOINs, because they are not supported in this example.

The `RIGHT JOIN` keyword would join all rows from the right table (`table2`) with the matching rows in the left table (`table1`). The result is NULL on the left side when there is no match.

The `FULL OUTER JOIN` keyword returns all rows from the left table (`table1`) and the right table (`table2`). This JOIN combines results from both `LEFT` and `RIGHT` JOINs and all information from both tables into one. You can imagine that this can involve lots of repetitious information and/or NULL values.

## Addtional Resources

---

These resources are a bit long-winded but are good for explaining `pandas` functions from a SQL programmer's perspective:

- [Pydata Video](https://www.youtube.com/watch?v=1uVWjdAbgBg)  
- [Associated GitHub Repo](https://github.com/gjreda/pydata2014nyc/tree/master/data)
- [`pandas` Merge, JOIN, and Concatenate](http://pandas.pydata.org/pandas-docs/stable/merging.html)